In [71]:
from asl.ARMParser import get_parser
from asl.ARMAST import Instruction
from ARMIntrinsics import EncodingFields
# from ARMTypes import *
from ARMAST import *
import json

with open("intr.json", 'r') as fi:
    I = json.load(fi)
with open("asl/arm_instrs.sexpr") as f:
    data = f.read()
parser = get_parser()
Instrs: List[Instruction] = parser.parse(data)


In [6]:
from ARMIntrinsics import EncodingFields


In [9]:
enc = "aarch64_vector_arithmetic_binary_uniform_add_wrapping_single_simd"
field = EncodingFields[enc]
op = None
encoding = None
for i in Instrs:
    for j in i.instEncodings:
        if j.encName == enc:
            op = i.instExecute
            encoding = j
            print(op, encoding)
            break


[StmtCall(qid=QualifiedIdentifier(archQualifier='ANY', id='CheckFPAdvSIMDEnabled64'), exprs=[]), StmtVarDeclInit(symboldecl=SymbolDecl(id='operand1', ty=TypeFun(id='bits', expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='datasize')))), expr=ExprIndex(expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='V')), slices=[SliceSingle(expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='n')))])), StmtVarDeclInit(symboldecl=SymbolDecl(id='operand2', ty=TypeFun(id='bits', expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='datasize')))), expr=ExprIndex(expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='V')), slices=[SliceSingle(expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='m')))])), StmtVarsDecl(ids=['result'], ty=TypeFun(id='bits', expr=ExprVarRef(qid=QualifiedIdentifier(archQualifier='ANY', id='datasize')))), StmtVarsDecl(ids=['element1'], ty=TypeFun(id='bits', expr=ExprVarRef(qid=QualifiedIdentifier(archQua

In [26]:
class BitVec(ASTNode):
    bv: str

    def __init__(self, bv: int):
        self.bv = bv

    def __repr__(self):
        return f"BitVec({self.bv})"


class Undefiend(ASTNode):
    def __repr__(self):
        return f"Undefiend()"


def ASTShrink(AST):
    if isinstance(AST, list):
        w = [ASTShrink(i) for i in AST]
        return [i for i in w if i is not None]
    if isinstance(AST, asl.StmtCall):
        return None  # Assume StmtCall useless
    elif isinstance(AST, asl.StmtVarDeclInit):
        return VarDeclInit(ASTShrink(AST.symboldecl), ASTShrink(AST.expr), GenUniqueID())
    elif isinstance(AST, asl.SymbolDecl):
        return VarsDecl([Var(AST.id, GenUniqueID())], ASTShrink(AST.ty), GenUniqueID())
    elif isinstance(AST, asl.StmtVarsDecl):
        return VarsDecl([Var(i, GenUniqueID()) for i in AST.ids], ASTShrink(AST.ty), GenUniqueID())
    elif isinstance(AST, asl.TypeFun):  # Only Returns Type width
        if AST.id == "bits":
            return ASTShrink(AST.expr)
        assert False
    elif isinstance(AST, asl.ExprVarRef):
        return Var(ASTShrink(AST.qid), GenUniqueID())
    elif isinstance(AST, asl.LValVarRef):
        return Var(ASTShrink(AST.qid), GenUniqueID())
    elif isinstance(AST, asl.LValArrayIndex):
        return ArrayIndex(ASTShrink(AST.lvalexpr), ASTShrink(AST.slices), GenUniqueID())
    elif isinstance(AST, asl.QualifiedIdentifier):
        return AST.id
    elif isinstance(AST, asl.ExprIndex):
        return ArrayIndex(ASTShrink(AST.expr), ASTShrink(AST.slices), GenUniqueID())
    elif isinstance(AST, asl.ExprSlice):
        return ArraySlice(ASTShrink(AST.expr), ASTShrink(AST.slices), GenUniqueID())
    elif isinstance(AST, asl.SliceSingle):
        return ASTShrink(AST.expr)
    elif isinstance(AST, asl.StmtFor):
        return For(Var(AST.id, GenUniqueID()), ASTShrink(AST.expr0), ASTShrink(AST.expr1), ASTShrink(AST.stmts), 1, GenUniqueID())
    elif isinstance(AST, asl.ExprLitInt):
        return Number(AST.integer)
    elif isinstance(AST, asl.ExprBinOp):
        return BinaryExpr(AST.binop[1:-1], ASTShrink(AST.lhs), ASTShrink(AST.rhs), GenUniqueID())
    elif isinstance(AST, asl.StmtAssign):
        return Update(ASTShrink(AST.lvalexpr), ASTShrink(AST.expr))
    elif isinstance(AST, asl.ExprCall):
        return Call(ASTShrink(AST.qid), ASTShrink(AST.exprs), GenUniqueID())
    elif isinstance(AST, asl.StmtIf):

        if AST.maybe_stmts != asl.Nothing():
            if len(AST.stmtifcases) == 1:
                return IfElse(ASTShrink(AST.stmtifcases[0].expr), ASTShrink(AST.stmtifcases[0].stmts), ASTShrink(AST.maybe_stmts), GenUniqueID())
            else:
                print(AST)
                assert False
        else:
            if len(AST.stmtifcases) == 1:
                return If(ASTShrink(AST.stmtifcases[0].expr), ASTShrink(AST.stmtifcases[0].stmts), GenUniqueID())
            else:
                print(AST)
                assert False
    elif isinstance(AST, asl.TypeRef):
        return 32
    elif isinstance(AST, asl.ExprLitBin):
        return BitVec(AST.bitvector)
    elif isinstance(AST, asl.StmtUndefined):
        return Undefiend()
    elif isinstance(AST, asl.ExprIf):
        return IfElse(ASTShrink(AST.exprtest), ASTShrink(AST.exprresult), ASTShrink(AST.exprelse), GenUniqueID())
    else:
        print(AST)
        assert False


newAST = ASTShrink(encoding.encDecode)


In [66]:

class DecodeContext:
    undefined = False

    def __init__(self, fields):
        self.fields = fields
        self.result = {}

    def symtable(self):
        # print(self.fields, self.result)
        return dict(self.fields, **self.result)

    def walkConstExprLV(self, AST: ASTNode):
        if isinstance(AST, Var):
            if AST.name.startswith("R"):
                assert False
            else:
                return AST.name
        else:
            print(AST)
            assert False

    def walkConstExprRV(self, AST: ASTNode):
        if isinstance(AST, Var):
            if AST.name.startswith("R"):
                return '00000'  # We just don't care about register
            else:
                return self.symtable()[AST.name]
        elif isinstance(AST, Call):
            if AST.funcname == "UInt":
                return int(self.walkConstExprRV(AST.args[0]), 2)
        elif isinstance(AST, BitVec):
            return AST.bv
        elif isinstance(AST, Number):
            return AST.val
        elif isinstance(AST, IfElse):
            cond = self.walkConstExprRV(AST.cond)
            assert type(cond) == bool
            if cond:
                return self.walkConstExprRV(AST.then)
            else:
                return self.walkConstExprRV(AST.otherwise)
        elif isinstance(AST, BinaryExpr):
            a = self.walkConstExprRV(AST.a)
            b = self.walkConstExprRV(AST.b)
            if AST.op == "<<":
                assert type(a) == int and type(b) == int
                return a << b
            elif AST.op == ">>":
                assert type(a) == int and type(b) == int
                return a >> b
            elif AST.op == "|":
                assert type(a) == bool and type(b) == bool
                return a | b
            elif AST.op == "&":
                assert type(a) == bool and type(b) == bool
                return a & b
            elif AST.op == "^":
                return a ^ b
            elif AST.op == "+":
                assert type(a) == int and type(b) == int
                return a + b
            elif AST.op == "-":
                assert type(a) == int and type(b) == int
                return a - b
            elif AST.op == "*":
                assert type(a) == int and type(b) == int
                return a*b
            elif AST.op == "DIV":
                assert type(a) == int and type(b) == int
                assert a % b == 0
                return a//b
            elif AST.op == ":":
                assert type(a) == str and type(b) == str
                return self.walkConstExprRV(AST.a) + self.walkConstExprRV(AST.b)
            elif AST.op == "==":
                return self.walkConstExprRV(AST.a) == self.walkConstExprRV(AST.b)
            else:
                print(AST)
                assert False
        else:
            print(AST)
            assert False

    def walkConstExprStmt(self, AST: ASTNode):
        if self.undefined:
            return
        if isinstance(AST, list):
            for i in AST:
                self.walkConstExprStmt(i)
        elif isinstance(AST, VarDeclInit):
            self.result[self.walkConstExprLV(
                AST.decl.ids[0])] = self.walkConstExprRV(AST.expr)
        elif isinstance(AST, If):
            if self.walkConstExprRV(AST.cond):
                self.walkConstExprStmt(AST.then)
        elif isinstance(AST, Undefiend):
            self.undefined = True
        else:
            print(AST)
            assert False


D = DecodeContext({'U': '0', 'size': '11', 'Rm': 'xxxxx',
                   'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx', 'Q': '1', })
D.walkConstExprStmt(newAST)
print(D.undefined)
print(D.result)


False
{'d': 0, 'n': 0, 'm': 0, 'esize': 64, 'datasize': 128, 'elements': 2, 'sub_op': False}


In [70]:
def genPossibleOpcode(fields):
    qwq = [fields]
    for i, v in fields.items():
        if i.startswith('R'):
            continue
        if 'x' in v:
            assert fields[i] == 'x'*len(fields[i])
            ln = len(fields[i])
            nqwq = []
            for z in range(2**ln):
                z = bin(z)[2:]
                nqwq = nqwq + [dict(q, **{i: z})for q in qwq]
            qwq = nqwq
    return qwq


opcd = genPossibleOpcode({'Q': 'x', 'U': '1', 'size': 'xx', 'Rm': 'xxxxx',
                          'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'})
print(len(opcd))
for i in opcd:
    D = DecodeContext(i)
    D.walkConstExprStmt(newAST)
    if D.undefined:
        print(i, 'undefined')
    else:
        print(i, D.result)


8
{'Q': '0', 'U': '1', 'size': '0', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'} {'d': 0, 'n': 0, 'm': 0, 'esize': 8, 'datasize': 64, 'elements': 8, 'sub_op': True}
{'Q': '1', 'U': '1', 'size': '0', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'} {'d': 0, 'n': 0, 'm': 0, 'esize': 8, 'datasize': 128, 'elements': 16, 'sub_op': True}
{'Q': '0', 'U': '1', 'size': '1', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'} {'d': 0, 'n': 0, 'm': 0, 'esize': 16, 'datasize': 64, 'elements': 4, 'sub_op': True}
{'Q': '1', 'U': '1', 'size': '1', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'} {'d': 0, 'n': 0, 'm': 0, 'esize': 16, 'datasize': 128, 'elements': 8, 'sub_op': True}
{'Q': '0', 'U': '1', 'size': '10', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx', 'Rd': 'xxxxx'} {'d': 0, 'n': 0, 'm': 0, 'esize': 32, 'datasize': 64, 'elements': 2, 'sub_op': True}
{'Q': '1', 'U': '1', 'size': '10', 'Rm': 'xxxxx', 'opcode': '10000', 'Rn': 'xxxxx